In [1]:
import sys
sys.path.append('..')
sys.path.append('../..')

In [2]:
import os
from os import path
import torch
from torch.utils.data import DataLoader
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms

In [3]:
from Pytorch.utils import Denormalize_tensor, Show_figure_with_bbox

In [4]:
exp_name = "COCO-DoubleCNNContextClassification-RN50+RN18-SGD"
epochs = 120
batch_size = 48
num_batch_train = 200
num_batch_val = 80
img_size = 256

In [5]:
parameters_path = path.join("parameters", exp_name)
plots_path = path.join("plots", exp_name)

In [6]:
import torch
from torch.utils.data import Dataset
from PIL import Image
from os import path
import json
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np

import cv2
from albumentations import Compose, Normalize, VerticalFlip, HorizontalFlip, Rotate, Resize, LongestMaxSize, PadIfNeeded, BboxParams
from albumentations.pytorch import ToTensor, ToTensorV2

from Pytorch.utils import Normalize_bbox_to_0_1, Convert_bbox_from_TLWH_to_TLBR, clamp

'''
CODING CONVENTIONS

- All names use underscore casing
- L_* for list variable names
- D_* for dictionary variable names
- T_* for tensor variable names
- M_* for Pytorch models
- DL_* for Data Loader
- DSET_* for Dataset
- Class names are in upper case
- Function names start with a capital letter
'''

AlbuToNormalizedTensor_transform = Compose(
    [
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

def AlbuToNormalizedTensor(image):
    return AlbuToNormalizedTensor_transform(image=image)['image']

class CONTEXT_DATASET(Dataset):
    def __init__(self,
                root_dirpath,
                images_dirpath,
                annotations_path,
                image_id_key="image_id",
                category_id_key="category_id",
                F_image_id_to_relative_path=None,
                augmentation=True,
                desired_size=224,
                batch_size=12,
                num_batch=1):

        self.root_dirpath = root_dirpath
        self.images_dirpath = images_dirpath
        self.annotations_path = annotations_path
        self.image_id_key=image_id_key
        self.category_id_key=category_id_key
        self.F_image_id_to_relative_path = F_image_id_to_relative_path
        self.augmentation = augmentation
        self.desired_size = desired_size
        self.batch_size = batch_size
        self.num_batch = num_batch

        with open(self.annotations_path) as f:
            self.D_instances = json.load(f)
            self.L_annotations = self.D_instances['annotations']
            self.L_categories = self.D_instances['categories']

        # list of real class indices in COCO dataset
        self.L_real_class_idx = [x['id'] for x in self.L_categories]
        # zero-based class indices of COCO dataset (0-79)
        self.L_class_idx = range(len(self.L_real_class_idx))
        # dict to map real class indices to zero-based indices
        self.D_real_class_idx_to_class_idx = { real_class_idx : class_idx for class_idx, real_class_idx in enumerate(self.L_real_class_idx)}

        # dict to map zero based indices to the relative annotations
        self.D_class_to_annotations = self.Create_class_to_annotations_dict()

        # filter zero-based class indices which have no annotations
        self.L_not_empty_class_idx = [class_idx for class_idx in self.L_class_idx if len(self.D_class_to_annotations[class_idx])>0]

        bbox_params = BboxParams(format='coco', label_fields=[self.category_id_key])
        self.augmentation_transform = Compose(
            [
                VerticalFlip(p=0.5),
                HorizontalFlip(p=0.5),
                LongestMaxSize(max_size=600),
                PadIfNeeded(min_height=710, min_width=710, border_mode=cv2.BORDER_CONSTANT),
                Rotate(p=1.0, limit=12, interpolation=cv2.INTER_LINEAR, border_mode=cv2.BORDER_CONSTANT),
                #Resize(height=desired_size, width=desired_size),
            ],
            bbox_params=bbox_params)
        
        self.last_resize_transform = Resize(height=desired_size, width=desired_size)

        self.loading_transform = Compose(
            [
                LongestMaxSize(max_size=desired_size),
                PadIfNeeded(min_height=desired_size, min_width=desired_size, border_mode=cv2.BORDER_CONSTANT),
            ],
            bbox_params=bbox_params)

        self.transform = self.augmentation_transform if self.augmentation else self.loading_transform

    def Create_class_to_annotations_dict(self):
        '''
        Create a dictionary of type (int : list) which map to each zero-based class index a list of
        all the COCO annotations of a specific class using self.D_real_class_idx_to_class_idx as map
        '''
        # init empty dictionary with zero-based class indices
        D_class_to_annotations = {idx : [] for idx in self.L_class_idx}

        for annotation in self.L_annotations:
            real_class_idx = annotation[self.category_id_key]
            class_idx = self.D_real_class_idx_to_class_idx[real_class_idx]
            D_class_to_annotations[class_idx].append(annotation)

        return D_class_to_annotations

    def Get_num_classes(self):
        return len(self.L_categories)

    def Get_class_idx_description(self, class_idx):
        return self.L_categories[class_idx]

    def __getitem__(self, index):
        try:
            # sample a random annotation from a random class
            item_class_idx = np.random.choice(self.L_not_empty_class_idx)
            D_item_annotation = np.random.choice(self.D_class_to_annotations[item_class_idx])

            item_image_id = D_item_annotation[self.image_id_key]

            if(self.F_image_id_to_relative_path):
                image_relative_path = self.F_image_id_to_relative_path(item_image_id)
            else:
                image_relative_path = item_image_id
            item_PIL_image = Image.open(path.join(self.images_dirpath, image_relative_path)).convert('RGB')
            image_width, image_height = item_PIL_image.size

            item_bbox = D_item_annotation['bbox']
            item_bbox[0] = clamp(item_bbox[0], 0, image_width)
            item_bbox[1] = clamp(item_bbox[1], 0, image_height)
            item_bbox[2] = clamp(item_bbox[2], 0, image_width-item_bbox[0])
            item_bbox[3] = clamp(item_bbox[3], 0, image_height-item_bbox[1])

            D_item_albumentation = {'image': np.array(item_PIL_image), 'bboxes': [item_bbox], self.category_id_key: [item_class_idx]}

            D_item_transformed = self.transform(**D_item_albumentation)
            if(len(D_item_transformed['bboxes'])==0):
                return self[0]

            # D_item_transformed['bboxes'] has got the only bbox we are passing in D_item_albumentation
            L_bbox_TLBR = Convert_bbox_from_TLWH_to_TLBR(list(D_item_transformed['bboxes'][0]))
            L_bbox_normalized = Normalize_bbox_to_0_1(L_bbox_TLBR, (710, 710))
            T_bbox_normalized = torch.Tensor(L_bbox_normalized)
            
            augmented_image = D_item_transformed["image"]
            L_bbox_TLBR = [int(x) for x in L_bbox_TLBR]
            cropped_image = augmented_image[L_bbox_TLBR[1]:L_bbox_TLBR[3], L_bbox_TLBR[0]:L_bbox_TLBR[2]]
            cropped_image = self.loading_transform(**{'image': cropped_image, 'bboxes': [], self.category_id_key: []})['image']
            augmented_image_resize=self.last_resize_transform(image=augmented_image)["image"]
            return (AlbuToNormalizedTensor(cropped_image), AlbuToNormalizedTensor(augmented_image_resize)), item_class_idx
        except:
            return self[0]

    def __len__(self):
        return self.num_batch*self.batch_size



In [ ]:
DSET_training = CONTEXT_DATASET(
    root_dirpath = '../../../Datasets/COCO',
    images_dirpath = '../../../Datasets/COCO/train2017',
    annotations_path = '../../../Datasets/COCO/annotations/instances_train2017.json',
    F_image_id_to_relative_path = lambda image_id: "{:012d}.jpg".format(image_id),
    augmentation = True,
    batch_size = batch_size,
    num_batch = num_batch_train,
    desired_size=img_size,
)

In [ ]:
DSET_validation = CONTEXT_DATASET(
    root_dirpath = '../../../Datasets/COCO',
    images_dirpath = '../../../Datasets/COCO/val2017',
    annotations_path = '../../../Datasets/COCO/annotations/instances_val2017.json',
    F_image_id_to_relative_path = lambda image_id: "{:012d}.jpg".format(image_id),
    augmentation = False,
    batch_size = batch_size,
    num_batch = num_batch_val,
    desired_size=img_size,
)

In [ ]:
denormalize_func = Denormalize_tensor(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))

In [ ]:
(T_img, T_context), class_idx = DSET_training[0]

In [ ]:
plt.imshow(transforms.ToPILImage()(denormalize_func(T_img)))

In [ ]:
plt.imshow(transforms.ToPILImage()(denormalize_func(T_context)))

In [ ]:
(T_img, T_context), class_idx = DSET_validation[0]

In [ ]:
plt.imshow(transforms.ToPILImage()(denormalize_func(T_img)))

In [ ]:
plt.imshow(transforms.ToPILImage()(denormalize_func(T_context)))

In [ ]:
DL_training = DataLoader(DSET_training, batch_size=batch_size, shuffle=False)
DL_validation = DataLoader(DSET_validation, batch_size=batch_size, shuffle=False)

In [ ]:
from torchvision.models import resnet50
from torchvision.models import resnet18

class M_CONTEXT_CLASSIFIER(torch.nn.Module):
    def __init__(self, num_classes, backbone=None):
        super(M_CONTEXT_CLASSIFIER, self).__init__()

        self.num_classes = num_classes
        self.M_context_cnn = resnet50(pretrained=True)
        self.M_context_cnn.fc = torch.nn.Linear(in_features=self.M_context_cnn.fc.in_features, out_features=1024, bias=True)
        
        self.M_patch_cnn = resnet50(pretrained=True)
        self.M_patch_cnn.fc = torch.nn.Linear(in_features=self.M_patch_cnn.fc.in_features, out_features=1024, bias=True)

        self.fc = torch.nn.Linear(in_features=self.M_patch_cnn.fc.out_features+self.M_context_cnn.fc.out_features, out_features=self.num_classes, bias=True)
        
    def forward(self, inputs):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        T_image, T_context = inputs
        T_extracted_features_patch = self.M_patch_cnn(T_image)
        T_extracted_features_context = self.M_context_cnn(T_context)
        T_features = torch.cat((T_extracted_features_patch, T_extracted_features_context), 1)
        return self.fc(T_features)

In [ ]:
M_classifier = M_CONTEXT_CLASSIFIER(num_classes=DSET_training.Get_num_classes())
M_classifier = M_classifier.to('cuda')

In [ ]:
def prepare_batch_fn(batch, gt):
    gt = gt.long()
    return batch, gt

In [ ]:
from torchtrainer.trainer import Trainer, Mode
from torchtrainer.callbacks.calculateaccuracycallback import CalculateTopNAccuracyCallback
from torchtrainer.callbacks.calculatelosscallback import CalculateLossCallback
from torchtrainer.callbacks.plotcallback import PlotCallback
from torchtrainer.callbacks.saveparameterscallback import SaveParametersCallback
from torchtrainer.callbacks.settqdmbardescription import SetTQDMBarDescription
from torchtrainer.callbacks.lrbatchschedulercallback import LRBatchSchedulerCallBack

In [ ]:
optimizer = torch.optim.SGD(M_classifier.parameters(), lr = 0.08, momentum = 0.1)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
trainer = Trainer(
    device = 'cuda',
    modes = [Mode.TRAIN, Mode.EVALUATE],
    model = M_classifier,
    data_loaders = {Mode.TRAIN : DL_training, Mode.EVALUATE : DL_validation},
    epochs = epochs,
    starting_epoch = 0,
    optimizer = optimizer,
    criterion = criterion,
    prepare_batch_fn = prepare_batch_fn,
    callbacks = [
        CalculateLossCallback(key='Loss'),
        CalculateTopNAccuracyCallback(keys=('Top-1 accuracy',), topk=(1,)),
        PlotCallback(plots_path, labels_map={Mode.TRAIN:"Train", Mode.EVALUATE:"Val"}, columns=['Loss', 'Top-1 accuracy']),
        SetTQDMBarDescription(keys=['Loss', 'Top-1 accuracy']),
        SaveParametersCallback(parameters_path),
    ]
)

In [ ]:
trainer.start()

In [ ]:
from torch_lr_finder import LRFinder

In [ ]:
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
optimizer = torch.optim.SGD(M_classifier.parameters(), lr=1e-7, momentum=0.0)
lr_finder = LRFinder(M_classifier, optimizer, criterion, device="cuda")
lr_finder.range_test(DL_training, end_lr=2, num_iter=100)
lr_finder.plot() # to inspect the loss-learning rate graph
lr_finder.reset() # to reset the model and optimizer to their initial state

In [ ]:
optimizer = torch.optim.SGD(M_classifier.parameters(), lr=1e-7, momentum=0.2)
lr_finder = LRFinder(M_classifier, optimizer, criterion, device="cuda")
lr_finder.range_test(DL_training, end_lr=2, num_iter=100)
lr_finder.plot() # to inspect the loss-learning rate graph
lr_finder.reset() # to reset the model and optimizer to their initial state

In [ ]:
optimizer = torch.optim.SGD(M_classifier.parameters(), lr=1e-7, momentum=0.6)
lr_finder = LRFinder(M_classifier, optimizer, criterion, device="cuda")
lr_finder.range_test(DL_training, end_lr=2, num_iter=100)
lr_finder.plot() # to inspect the loss-learning rate graph
lr_finder.reset() # to reset the model and optimizer to their initial state